In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from utils.io_func import load_from_npy, load_from_csv

In [ ]:
TIME_STEP_LIST = [2, 4, 6, 7, 8, 10, 13, 16, 19, 22]
GROUND_TRUTH_PATH = "../preprocessing/out/Site_1/y-corn_soybean-2018.npy"
PRED_DIR_TEMPLATE = (
    "../experiments/out/in_season/AtLSTM-corn_soybean/time_step_{time_step}/"
)
HARD_PRED_PATH_TEMPLATE = os.path.join(
    PRED_DIR_TEMPLATE, "y_test_hard_pred.csv"
)
SOFT_PRED_PATH_TEMPLATE = os.path.join(
    PRED_DIR_TEMPLATE, "y_test_soft_pred.csv"
)

# Input

In [ ]:
y_test = load_from_npy(GROUND_TRUTH_PATH)
y_test_hard_pred_list, y_test_soft_pred_list = [], []
for time_step in TIME_STEP_LIST:
    y_test_hard_pred_list.append(load_from_csv(
        HARD_PRED_PATH_TEMPLATE.format(time_step=time_step)
    ).values.flatten())
    y_test_soft_pred_list.append(load_from_csv(
        SOFT_PRED_PATH_TEMPLATE.format(time_step=time_step)
    ).values)

# Predicted score analysis

In [ ]:
class_names = ["Corn", "Soybean"]
class_values = [0, 1]

for i, class_name in enumerate(class_names):
    class_value = class_values[i]
    class_sample_index = (y_test == class_value)
    plt.figure(figsize=(18, 6))
    plt.subplots_adjust(wspace=0.6, hspace=0.85)
    for j, time_step in enumerate(TIME_STEP_LIST):
        predicted_score = y_test_soft_pred_list[j][class_sample_index][
            :, class_value
        ]  # class value is equal to class index (0: 0, 1: 1)
        plt.subplot(2, 5, j + 1)
        plt.hist(
            predicted_score,
            weights=np.ones_like(predicted_score)/predicted_score.shape[0], bins=10,
            color="#9AC8F1", edgecolor="#3F78CB", linewidth=2
        )
        plt.xlabel("Predicted score")
        plt.ylabel("Frequency")
        plt.ylim([0, 1])
        plt.yticks([0, 0.2, 0.4, 0.6, 0.8, 1])
        plt.title(
            "Week {}".format(TIME_STEP_LIST[j]),
            pad=plt.rcParams["axes.titlepad"]*2
        )
        avg = predicted_score.mean()
        plt.plot([avg, avg], [0, 1], color="#F20923", linestyle="--")
        plt.annotate(
            "{:.2f}".format(avg), xy=(avg, 1.03), xycoords="axes fraction",
            va="bottom", ha="center", color="#F20923"
        )
        ax = plt.gca()
        ax.yaxis.grid(True)